<a href="https://colab.research.google.com/github/Hamoda-dabbit/Mining---classification-in-Arabic-Article/blob/main/CNN/XLNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# transformers - SOTA implementation of pretrained models
!pip install -U simpletransformers

In [9]:
# Importing libraries
import warnings
warnings.filterwarnings("ignore")
import nltk
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from nltk.stem.isri import ISRIStemmer
import string
from sklearn.utils import shuffle
import numpy as np
import re
import pandas as pd

In [ ]:
df = pd.read_excel("drive/MyDrive/mining/1000.xlsx")

df = shuffle(df)
df

In [3]:
print(np.sum(df.isnull().any(axis=1)))

clean_df= df.dropna()
print(np.sum(clean_df.isnull().any(axis=1)))
#-------------
print (df.notnull().any(axis = 0))
print(len(df))
#-------------
def remove_hashtag(df, col = 'text'):
    for letter in r'#.][!XR':
      df[col] = df[col].astype(str).str.replace(letter,'', regex=True)
        
remove_hashtag(clean_df)
clean_df
#-------------
arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations

def remove_punctuations(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)
#-------------
def normalize_arabic(text):
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)
    return text
#-------------    
def remove_repeating_char(text):
    return re.sub(r'(.)\1+', r'\1', text)
#-------------
def processDocument(doc, stemmer): 

    #Replace @username with empty string
    doc = re.sub(r'@[^\s]+', ' ', doc)
    doc = re.sub(r'_', ' ', doc)
    doc = re.sub(r'\n', ' ', doc)
    doc = re.sub(r'[a-z,A-Z]', '', doc)
    doc = re.sub(r'\d', '', doc)
    #Convert www.* or https?://* to " "
    doc = re.sub('((www\.[^\s]+)|(https?://[^\s]+))',' ',doc)
    #Replace #word with word
    doc = re.sub(r'#([^\s]+)', r'\1', doc)
    # remove punctuations
    doc= remove_punctuations(doc)
    # normalize the tweet
    doc= normalize_arabic(doc)
    # remove repeated letters
    doc=remove_repeating_char(doc)
    #stemming
    doc = stemmer.stem(doc)
    
    return doc

stemmer = ISRIStemmer()
clean_df["text"] = clean_df['text'].apply(lambda x: processDocument(x, stemmer))
print(clean_df)

12
0
text     True
targe    True
dtype: bool
5000
                                                   text  targe
3582  رغم من اكتساحه لعد من المدن الكبري في جل مناطق...      3
1934  تمكنت مصالح ولايه امن مراكش اول امس الثلاثاء ب...      1
3283  دعا حزب الاتحاد الاشتراكي لقوات الشعبيه الي هي...      3
851   عبدالاله بوسحابه اخبارنا المغربيه بدعوه من الس...      0
1877  وضعت المصالح الامنيه بفاس شخصا من ذوي السوابق ...      1
...                                                 ...    ...
4973  اخبارنا المغربيه محمد الميموني بعد مضي سنوات ك...      4
1593  وكيل العام لملك اعطي اوامره بتعميق البحث في ال...      1
494   اعلن مصطفي الخلفي وزير الاتصال الناطق الرسمي ب...      0
2473  علي عد ايام من انطلاق الدوره المقبله لمؤتمر ال...      2
2618  قال رئيس الفدراليه الوطنيه لكهرباء والالكتروني...      2

[4988 rows x 2 columns]


In [4]:
clean_df.targe.value_counts()

3    1000
1    1000
0    1000
4    1000
2     988
Name: targe, dtype: int64

In [5]:
# Total dataset length

X = clean_df["text"]
y = clean_df['targe']


dataset_length = df.shape[0]

# Training dataset length
split = int(dataset_length * 0.25)
split
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

train_df = pd.DataFrame(X_train)
train_df['target'] = y_train

eval_df = pd.DataFrame(X_test)
eval_df['target'] = y_test

In [6]:
eval_df


,text,target
4317,اخبارنا المغربيه الرباط خلق قرار الجنه المنظمه...,4
830,قضت الدائره مدني بمحكمه جنوب القاهره بقبول الد...,0
853,سطات عادل النويتي قضت المحكمه الابتدائيه بمدين...,0
2772,لم يمهل الصندوق الوطني لضمان الاجتماعي الكثير ...,2
3537,اعتبر الملك محمد السادس مساء اليوم السبت ان ال...,3
...,...,...
4973,اخبارنا المغربيه محمد الميموني بعد مضي سنوات ك...,4
1593,وكيل العام لملك اعطي اوامره بتعميق البحث في ال...,1
494,اعلن مصطفي الخلفي وزير الاتصال الناطق الرسمي ب...,0
2473,علي عد ايام من انطلاق الدوره المقبله لمؤتمر ال...,2


In [7]:
from simpletransformers.classification import ClassificationModel
import pandas as pd
import logging
import sklearn


logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# They are lot of arguments to play with
'''
args = {
   'output_dir': 'outputs/',
   'cache_dir': 'cache/',
   'fp16': True,
   'fp16_opt_level': 'O1',
   'max_seq_length': 256,
   'train_batch_size': 8,
   'eval_batch_size': 8,
   'gradient_accumulation_steps': 1,
   'num_train_epochs': 3,
   'weight_decay': 0,
   'learning_rate': 4e-5,
   'adam_epsilon': 1e-8,
   'warmup_ratio': 0.06,
   'warmup_steps': 0,
   'max_grad_norm': 1.0,
   'logging_steps': 50,
   'evaluate_during_training': False,
   'save_steps': 2000,
   'eval_all_checkpoints': True,
   'use_tensorboard': True,
   'overwrite_output_dir': True,
   'reprocess_input_data': False,
}
'''

# Create a ClassificationModel
model = ClassificationModel('xlnet', 'xlnet-base-cased',num_labels=5, args={'num_train_epochs':1, 'train_batch_size':32, 'max_seq_length':128},use_cuda=False) # You can set class weights by using the optional weight argument

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.bias', 'sequence_summary.summary.weight', 'logits_proj.weight', 'sequence_summary.summary.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

In [ ]:
train_df

In [ ]:

# Train the model
model.train_model(train_df)

  0%|          | 0/1250 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/40 [00:00<?, ?it/s]

(40, 1.624020367860794)

In [ ]:

# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(eval_df, acc=sklearn.metrics.accuracy_score)

  0%|          | 0/3738 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/468 [00:00<?, ?it/s]

In [ ]:
result

{'acc': 0.20224719101123595,
 'eval_loss': 1.6127002715555012,
 'mcc': 7.614484951944167e-05}

In [ ]:
print(result)

{'mcc': 7.614484951944167e-05, 'acc': 0.20224719101123595, 'eval_loss': 1.6127002715555012}
